In [9]:
import csv

with open('../../datasets/adversarial_layer/structured_py_processed_data.csv', 'w') as wf:

    fieldnames = ['prompt', 'completion']
    writer = csv.DictWriter(wf, fieldnames=fieldnames)
    writer.writeheader()

    with open('../../datasets/adversarial_layer/structured_raw_data.csv', 'r') as rf:
        reader = csv.reader(rf, delimiter=',')
        line_count = 0
        for row in reader:
            if line_count == 0:
                print(f'Column names are {", ".join(row)}')
                line_count += 1
            else:
                processed_prompt = "instruction: Given the prompt and the completion, find the mistake in the completion, if any. If mistake found, propose a fix." + "\n---\n" + "prompt: \"\"\"" + row[0] + "\"\"\"" + "\n---\n" + "completion: \"\"\"" + row[1] + "\"\"\"" + "\n---\n" + "mistake:"
                processed_completion = "\"\"\"" + row[2] + "\"\"\"" + "\n---\n" + "fix: \"\"\"" + row[3] + "\"\"\"" + "\n---\n" + "XXX_END_GEN_XXX"
                writer.writerow({
                    'prompt': processed_prompt,
                    'completion': processed_completion
                })
                line_count += 1

Column names are prompt, completion, mistake, fix, , 
